In [163]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, Button, HoverTool, Select, TapTool, Label
from bokeh.palettes import brewer
from bokeh.layouts import layout, widgetbox, row, column
from bokeh.models.glyphs import Patches


import holoviews as hv
import sys, os
import re

# WYKRES - geolokalizacja

### Wczytanie zbiorów

* Dane o np. o gestości zaludnienia na 1km^2 w woj. mazowieckim w latach 2008-2018, GUS: 
https://bdl.stat.gov.pl/BDL/dane/podgrup/wymiary
* Dane o położeniu powiatów (geolokalizacyjne) w Polsce: http://www.gugik.gov.pl/pzgik/dane-bez-oplat/dane-z-panstwowego-rejestru-granic-i-powierzchni-jednostek-podzialow-terytorialnych-kraju-prg (jednostki_administracyjne.zip --> pliki .shp)

Pliki znajdują się w folderze ../data/jednostki_administracyjne/ i ../data/dane_gus/

bokeh na podstawie: https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
opcja play (HoloViews) na podstawie: http://holoviews.org/reference/apps/bokeh/player.html

In [164]:
def TERYT_powiat(x):
    if len(str(x))>=4:     
        if str(x)[3]=='0': 
            return '0' + str(x)[0:3] 
        else: 
            return str(x)[0:4]
    else: return str(x)[0:4]

* geolokalizacyjne pliki

In [165]:
shapefile = os.path.realpath('jednostki_administracyjne/Powiaty.shp')
#Read shapefile using Geopandas
pdf = gpd.read_file(shapefile,  encoding='utf-8')[['JPT_KOD_JE', 'JPT_NAZWA_', 'geometry']]
pdf.columns = ['TERYT', 'powiat', 'geometry']
pdf.TERYT = pdf.TERYT.apply(lambda x: TERYT_powiat(x) ) ### !!!!! pierwsze 4 (powiat)

In [166]:
pdf.head(10)

,TERYT,powiat,geometry
0,1815,powiat ropczycko-sędziszowski,"POLYGON ((21.69428632700004 50.17849213600005,..."
1,0141,powiat łosicki,"POLYGON ((22.72380967100003 52.28038668500005,..."
2,1418,powiat piaseczyński,"POLYGON ((21.11009714700003 52.11316839000006,..."
3,1425,powiat radomski,"POLYGON ((20.98914935600004 51.48946604800005,..."
4,1427,powiat sierpecki,"POLYGON ((19.50875873300004 52.86784480400007,..."
5,0143,powiat szydłowiecki,"POLYGON ((20.80150421600007 51.31373953500008,..."
6,1433,powiat węgrowski,"POLYGON ((21.71293380800006 52.63281474900003,..."
7,1404,powiat gostyniński,"POLYGON ((19.61602260800004 52.28815325300008,..."
8,1405,powiat grodziski,"POLYGON ((20.70416261500003 52.13281420900006,..."
9,0611,powiat łukowski,"POLYGON ((22.43352922100007 52.01743078600003,..."


* dane z GUS + połączenie z plikami geolokalizacyjnymi

In [167]:
dane_p = pd.DataFrame()
zb = 'wynagrodzenie_srednie_brutto'
path = 'dane_gus/powiaty/' + zb + '.csv'
datafile = os.path.realpath(path) 
#Read csv file using pandas
df = pd.read_csv(datafile, sep=';', decimal=',')
df.columns=['TERYT', 'Nazwa', zb +'_' + '2008',  zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', zb +'_' + '2018', '']
df = df.drop(columns='')
df.TERYT =df.TERYT.apply(lambda x: TERYT_powiat(x) ) ### !!!!! pierwsze 4 (powiat)

dane_p = df   

In [168]:
zbiory = [ 'nowo_zarejestr_podm_sekt_pryw', 
          'nowe_sekt_pryw_sp_handlowe', 
          'nowe_sekt_publ_sp_handlowe', 
          'podm_w_rej_reg_na10tys_ludn', 
          'udzial_wyrejestr_do_nowych', 
          'podm_w_rej_reg_sekt_pryw_HANDL', 
          'bud_niemieszk_ogolem', 
          'magazyny_silosy_zb',
         'drogi_utwardzone_km', 
         'drogi_utwardzone_km_na100km2', 
         'gestosc_zaludnienia_na1km2']
   

In [169]:
for zb in zbiory:
    path = 'dane_gus/powiaty/' + zb + '.csv'
    datafile = os.path.realpath(path) 
    #Read csv file using pandas
    df = pd.read_csv(datafile, sep=';', decimal=',')
    if len(df.columns) == 13: #drogi_utwardzone_km
        df.columns=['TERYT', 'Nazwa', zb +'_' + '2008',  zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', '']
    else:
        df.columns=['TERYT', 'Nazwa', zb +'_' + '2008',  zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', zb +'_' + '2018', '']
    df = df.drop(columns='')
    df.TERYT =df.TERYT.apply(lambda x: TERYT_powiat(x))   
   
    dane_p = pd.merge(dane_p, df, on=['TERYT', 'Nazwa'], how='outer')

In [170]:
dane_p.head(10)

,TERYT,Nazwa,wynagrodzenie_srednie_brutto_2008,wynagrodzenie_srednie_brutto_2009,wynagrodzenie_srednie_brutto_2010,wynagrodzenie_srednie_brutto_2011,wynagrodzenie_srednie_brutto_2012,wynagrodzenie_srednie_brutto_2013,wynagrodzenie_srednie_brutto_2014,wynagrodzenie_srednie_brutto_2015,...,gestosc_zaludnienia_na1km2_2009,gestosc_zaludnienia_na1km2_2010,gestosc_zaludnienia_na1km2_2011,gestosc_zaludnienia_na1km2_2012,gestosc_zaludnienia_na1km2_2013,gestosc_zaludnienia_na1km2_2014,gestosc_zaludnienia_na1km2_2015,gestosc_zaludnienia_na1km2_2016,gestosc_zaludnienia_na1km2_2017,gestosc_zaludnienia_na1km2_2018
0,0,POLSKA,3158.48,3315.38,3435.00,3625.21,3744.38,3877.43,4003.99,4150.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0200,DOLNOŚLĄSKIE,3135.83,3295.44,3412.37,3587.25,3709.32,3868.86,4042.86,4204.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0201,Powiat bolesławiecki,2476.41,2629.01,2789.80,2936.92,3057.28,3231.49,3422.90,3548.69,...,68.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0
3,0202,Powiat dzierżoniowski,2489.40,2645.13,2832.05,3024.11,3113.13,3156.11,3299.96,3406.18,...,217.0,222.0,221.0,220.0,219.0,217.0,216.0,215.0,213.0,212.0
4,0203,Powiat głogowski,2647.38,2859.95,2914.21,3028.65,3122.99,3320.48,3460.12,3615.99,...,198.0,204.0,204.0,204.0,204.0,204.0,203.0,203.0,203.0,202.0
5,0204,Powiat górowski,2376.54,2464.05,2598.06,2755.49,2908.49,3066.83,3142.52,3242.71,...,49.0,50.0,50.0,50.0,49.0,49.0,49.0,48.0,48.0,48.0
6,0205,Powiat jaworski,2368.60,2506.86,2654.67,2837.23,2983.89,3040.71,3142.45,3323.37,...,89.0,91.0,90.0,90.0,90.0,89.0,88.0,88.0,88.0,87.0
7,0206,Powiat jeleniogórski,2369.63,2562.15,2699.92,2867.09,3031.43,3128.87,3232.50,3413.66,...,102.0,104.0,104.0,104.0,104.0,104.0,103.0,103.0,102.0,102.0
8,0207,Powiat kamiennogórski,2374.82,2546.98,2701.25,2864.01,2983.93,3045.35,3161.57,3269.22,...,115.0,116.0,116.0,115.0,114.0,113.0,112.0,112.0,111.0,110.0
9,0208,Powiat kłodzki,2511.35,2663.41,2803.23,2962.33,3063.43,3200.48,3333.36,3410.23,...,100.0,102.0,102.0,101.0,100.0,100.0,99.0,98.0,98.0,97.0


### obsluga wyjatkow

In [171]:
dane_p.loc[dane_p.TERYT == '0620', 'TERYT'] = '0062'
dane_p.loc[dane_p.TERYT == '0610', 'TERYT'] = '0061'
dane_p.loc[dane_p.TERYT == '0410', 'TERYT'] = '0041'
dane_p.loc[dane_p.TERYT == '0210', 'TERYT'] = '0021'
dane_p.loc[dane_p.TERYT == '0220', 'TERYT'] = '0022'
dane_p.loc[dane_p.TERYT == '0810', 'TERYT'] = '0081'

## Wykres

In [ ]:
renderer = hv.renderer('bokeh')

In [ ]:
# Declare the HoloViews object
start = 2008
end = 2018

In [80]:
def animate_update():
    year = slider.value
    if year < end:
        year = slider.value + 1
        slider.value = year
    
    label = Label(x=1.1, y=18, text=str(year), text_font_size='70pt', text_color='#eeeeee')
    p.add_layout(label)
            

def animate():
    if button.label == '► Play' and slider.value<end:
        button.label = '❚❚ Pause'
        curdoc().add_periodic_callback(animate_update, 10000) ## co 1000 mili sec
    else:
        print('$$$$$$$$$$$ zakonczone $$$$$$$$$$')
        button.label = '► Play'
        curdoc().remove_periodic_callback(animate_update)
        slider.value==2008

In [ ]:
#Define function that returns json_data for year selected by user.
    
def json_data():
    
    df = dane_p
    
    #Merge gus dataframes pdf 
    merged = pdf.set_index('TERYT').merge(df.set_index('TERYT'), on='TERYT', how='left') 
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data())
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 6000, nan_color = '#d9d9d9')

#Value to plot
col='wynagrodzenie_srednie_brutto_2008'

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     location = (0,0), orientation = 'horizontal') #, major_label_overrides = tick_labels)

TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover,tap'

#Create figure object.
p = figure(title ='Wynagrodzenie_srednie_brutto w 2008r.', plot_height = 900 , plot_width = 900, tools = TOOLS)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :col, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1) #, hover_fill_color="firebrick")

#Add hover tool
p.select_one(HoverTool).tooltips = [
                                    ('powiat',' @powiat'),('wartosc', '@' + col)
                                ]


#Specify layout
p.add_layout(color_bar, 'below')
# Define the callback function: update_plot
def update_plot(attr, old, new):
    print('wybrany rok: ', slider.value)
    print('wybrany zb: ', select.value)
    yr = slider.value
    zb = select.value
    
    col = select.value+'_' + str(slider.value)
    p.select_one(HoverTool).tooltips = [
                                        ('powiat',' @powiat'),('wartosc', '@' + col)
                                    ]

    p.patches('xs','ys', source = geosource,fill_color = {'field' :col, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
    
    if select.value == 'udzial_wyrejestr_do_nowych':
        p.title.text = 'Udział wyrejestrowanych podmiotów do zarejestrowanych [%] w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=20
        
    elif select.value == 'podm_w_rej_reg_na10tys_ludn':
        p.title.text = 'Liczba podmiotów wpisanych do rej. REGON na 10tys. ludności - stan na %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=3000
        
    elif select.value == 'nowe_sekt_publ_sp_handlowe':
        p.title.text =  'Liczba nowo zarejestr. podmiotów HANDLOWYCH w sekt. publicznym w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=10
    
    elif select.value == 'nowe_sekt_pryw_sp_handlowe':
        p.title.text = 'Liczba nowo zarejestr. podmiotów HANDLOWYCH w sekt. prywatnym w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=500
    
    elif select.value == 'nowo_zarejestr_podm_sekt_pryw':
        p.title.text = 'Liczba nowo zarejestrowanych podmiotow w sektorze prywatnym w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=3000
    
    
    elif select.value == 'wynagrodzenie_srednie_brutto':
        p.title.text = 'Wynagrodzenie_srednie_brutto w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=6000
        
    elif select.value == 'podm_w_rej_reg_sekt_pryw_HANDL':
        p.title.text = 'Podmioty zarejstrowane wg REGON, sekt. pryw. HANDLOWE, %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=15000   
        
        
    elif select.value ==  'bud_niemieszk_ogolem':
        p.title.text = 'Powierzchnia użytkowa budynków niemieszkalnych ogółem [m2], oddane do użytkowania w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=300000   
        
    elif select.value ==  'magazyny_silosy_zb':
        p.title.text = 'Powierzchnia użytkowa magazynów/silosów/zbiorników ogółem [m2], oddane do użytkowania w %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=200000
        
    elif select.value ==  'drogi_utwardzone_km':
        p.title.text = 'Łączna długość drug utwardzonych [km] , %d r.' %yr
        # update the existing transform
        color_mapper.low=0
        color_mapper.high=1000   
   
    elif select.value ==  'drogi_utwardzone_km_na100km2':
            p.title.text = 'Łączna długość drug utwardzonych na 100km2,  [km] , %d r.' %yr
            # update the existing transform
            color_mapper.low=0
            color_mapper.high=400   
        
        
    elif select.value ==  'gestosc_zaludnienia_na1km2':
            p.title.text = 'Gęstość zaludnienia na 1km2, %d r.' %yr
            # update the existing transform
            color_mapper.low=0
            color_mapper.high=1000      
    
    color_bar.trigger('change')

    
# Make a slider object: slider 
slider = Slider(title = 'Rok',start = 2008, end = 2018, step = 1, value = 2008)
slider.on_change('value', update_plot)


zbiory2 = [ 'nowo_zarejestr_podm_sekt_pryw', 
          'nowe_sekt_pryw_sp_handlowe', 
          'nowe_sekt_publ_sp_handlowe', 
          'podm_w_rej_reg_na10tys_ludn', 
          'udzial_wyrejestr_do_nowych', 
          'podm_w_rej_reg_sekt_pryw_HANDL', 
          'bud_niemieszk_ogolem', 
          'magazyny_silosy_zb',
         'drogi_utwardzone_km', 
         'drogi_utwardzone_km_na100km2', 
         'gestosc_zaludnienia_na1km2',
         'wynagrodzenie_srednie_brutto']




# Make select box
select = Select(title="Prezentowana wartość:", value="wynagrodzenie_srednie_brutto", options=zbiory2)
select.on_change('value', update_plot)




# Button for animation
button = Button(label='► Play', width=60)
button.on_click(animate)


# Make a column layout of widgetbox(slider) and plot, and add it to the current document
#layout = row(p,widgetbox(slider), widgetbox(select), widgetbox(button))
#curdoc().add_root(layout)
#Display plot inline in Jupyter notebook
#output_notebook()
#Display plot
#show(layout)


# WYKRES - liniowy

na podstawie artykułu : https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/
        Dotyczy analizy Vector AR models (mutliple time series)

### Wczytanie zbiorów

In [172]:
def Kod_powiat(x):
    if len(str(x))>=4:     
        if str(x)[3]=='0': 
            return '0' + str(x)[0:3] 
        else: 
            return str(x)[0:4]
    else: return str(x)[0:4]

In [173]:
def wyciagnij_rok(x):
    match = re.search(r"(Kod|Nazwa|20[0-9]{2,2})", x)
    if match is None:
        return None
    else:
        return match.group(0)

In [174]:
dane_p_liniowy = pd.DataFrame()
zb = 'wynagrodzenie_srednie_brutto'
path = 'dane_gus/powiaty_analiza/' + zb + '.csv'
datafile = os.path.realpath(path) 
#Read csv file using pandas
df = pd.read_csv(datafile, sep=';', decimal=',')
df.columns = pd.Series(df.columns).apply(lambda x:  wyciagnij_rok(x))
df = df.drop(columns=['Nazwa'])
df.Kod = df.Kod.apply(lambda x: Kod_powiat(x))   
df = df.melt(id_vars=["Kod"], var_name="Rok", value_name=zb)
df['Rok'] = df.Rok

dane_p_liniowy = df   

In [175]:
zbiory = [ 'nowo_zarejestr_podm_sekt_pryw', 
          'nowe_sekt_pryw_sp_handlowe', 
          'nowe_sekt_publ_sp_handlowe', 
          'podm_w_rej_reg_na10tys_ludn', 
          'udzial_wyrejestr_do_nowych', 
          'podm_w_rej_reg_sekt_pryw_HANDL', 
          'bud_niemieszk_ogolem', 
          'magazyny_silosy_zb']

In [176]:
for zb in zbiory:
    path = 'dane_gus/powiaty_analiza/' + zb + '.csv'
    datafile = os.path.realpath(path) 
    #Read csv file using pandas
    df = pd.read_csv(datafile, sep=';', decimal=',')
    df.columns = pd.Series(df.columns).apply(lambda x:  wyciagnij_rok(x))
    df = df.drop(columns=['Nazwa'])
    df.Kod = df.Kod.apply(lambda x: Kod_powiat(x))   
    df = df.melt(id_vars=["Kod"], var_name="Rok", value_name=zb)
    dane_p_liniowy = pd.merge(dane_p_liniowy, df, on=['Kod', 'Rok'], how='left')

In [177]:
dane_p_liniowy.groupby('Rok').mean()

,wynagrodzenie_srednie_brutto,nowo_zarejestr_podm_sekt_pryw,nowe_sekt_pryw_sp_handlowe,nowe_sekt_publ_sp_handlowe,podm_w_rej_reg_na10tys_ludn,udzial_wyrejestr_do_nowych,podm_w_rej_reg_sekt_pryw_HANDL,bud_niemieszk_ogolem,magazyny_silosy_zb
Rok,,,,,,,,,
2002,1892.033224,NaN,NaN,NaN,802.176322,NaN,NaN,NaN,NaN
2003,1957.381363,614.730973,23.746903,0.554080,861.738053,4.488496,NaN,NaN,NaN
2004,2037.486088,600.088496,25.300885,0.499051,857.375221,6.703363,NaN,NaN,NaN
2005,2117.944054,672.429544,17.962284,0.320150,930.195914,6.989157,453.970141,16784.066527,1247.284966
2006,2221.043316,722.266108,20.573599,0.172100,926.410162,9.126925,471.252488,27297.613934,3069.275013
2007,2417.645317,677.189104,20.567837,0.043827,942.319015,6.819015,485.938711,20841.853850,3297.259822
2008,2652.025993,741.182294,21.533787,0.625869,964.127816,6.885333,496.000524,21105.111577,3414.947093
2009,2817.118287,835.054479,21.013620,0.332977,953.877947,11.194185,512.421163,23042.064955,3365.203772
2010,2879.167150,943.741750,26.595600,0.477819,981.708748,7.038921,533.645888,32115.440021,2375.157674


wybor lat w ktorych mamy pełny zakres danych i usuniecie pustych wierszy na koncu ( Rok is null)

In [178]:
dane_p_liniowy = dane_p_liniowy[~dane_p_liniowy.Rok.isin(['2002', '2003', '2004', '2018'])]

In [179]:
dane_p_liniowy = dane_p_liniowy[~pd.isnull(dane_p_liniowy.Rok)]

In [180]:
def dodaj_powiat_nazwa(dane_p_liniowy):
    zb = 'wynagrodzenie_srednie_brutto'
    path = 'dane_gus/powiaty/' + zb + '.csv'
    datafile = os.path.realpath(path) 
    #Read csv file using pandas
    df = pd.read_csv(datafile, sep=';', decimal=',')
    #df.columns=['Kod', 'Nazwa', zb +'_' + '2008', zb +'_' +  '2009', zb +'_' +  '2010',  zb +'_' + '2011',  zb +'_' + '2012', zb +'_' + '2013', zb +'_' + '2014', zb +'_' + '2015', zb +'_' + '2016', zb +'_' + '2017', zb +'_' + '2018', '']
    df = df[['Kod', 'Nazwa']]
    df.Kod =df.Kod.apply(lambda x: Kod_powiat(x))  
    dane_p_liniowy = pd.merge(dane_p_liniowy, df, on=['Kod'], how='left')
    return dane_p_liniowy

In [181]:
dane_p_liniowy = dodaj_powiat_nazwa(dane_p_liniowy)

In [182]:
dane_p_liniowy

,Kod,Rok,wynagrodzenie_srednie_brutto,nowo_zarejestr_podm_sekt_pryw,nowe_sekt_pryw_sp_handlowe,nowe_sekt_publ_sp_handlowe,podm_w_rej_reg_na10tys_ludn,udzial_wyrejestr_do_nowych,podm_w_rej_reg_sekt_pryw_HANDL,bud_niemieszk_ogolem,magazyny_silosy_zb,Nazwa
0,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,96.0,Powiat bolesławiecki
1,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,96.0,Powiat siemiatycki
2,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,966.0,Powiat bolesławiecki
3,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,966.0,Powiat siemiatycki
4,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,96.0,Powiat bolesławiecki
5,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,96.0,Powiat siemiatycki
6,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,966.0,Powiat bolesławiecki
7,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,966.0,Powiat siemiatycki
8,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,46.0,36634.0,96.0,Powiat bolesławiecki
9,0201,2005,2086.26,615.0,11.0,1.0,790.0,6.5,46.0,36634.0,96.0,Powiat siemiatycki


In [183]:
#df['Rok'] = pd.to_datetime(df.Rok , format = '%Y')
dane_p_liniowy.index = dane_p_liniowy.Rok
dane_p_liniowy = dane_p_liniowy.drop(['Rok'], axis=1)

In [184]:
dane_p_liniowy

,Kod,wynagrodzenie_srednie_brutto,nowo_zarejestr_podm_sekt_pryw,nowe_sekt_pryw_sp_handlowe,nowe_sekt_publ_sp_handlowe,podm_w_rej_reg_na10tys_ludn,udzial_wyrejestr_do_nowych,podm_w_rej_reg_sekt_pryw_HANDL,bud_niemieszk_ogolem,magazyny_silosy_zb,Nazwa
Rok,,,,,,,,,,,
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,96.0,Powiat bolesławiecki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,96.0,Powiat siemiatycki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,966.0,Powiat bolesławiecki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,36634.0,966.0,Powiat siemiatycki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,96.0,Powiat bolesławiecki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,96.0,Powiat siemiatycki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,966.0,Powiat bolesławiecki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,313.0,7427.0,966.0,Powiat siemiatycki
2005,0201,2086.26,615.0,11.0,1.0,790.0,6.5,46.0,36634.0,96.0,Powiat bolesławiecki


In [185]:
powiaty_lista = dane_p_liniowy.Nazwa.unique()

In [186]:
def wybor_powiatu(powiat):
    data = dane_p_liniowy[dane_p_liniowy.Nazwa == powiat]
    data = data.drop(columns=['Kod', 'Nazwa'])
    #data.index = pd.Series(data.index).apply(lambda x: x.year)
    return data

In [187]:
powiat = 'Powiat poznański'

In [188]:
data = wybor_powiatu(powiat)

In [189]:
data.head(10)

,wynagrodzenie_srednie_brutto,nowo_zarejestr_podm_sekt_pryw,nowe_sekt_pryw_sp_handlowe,nowe_sekt_publ_sp_handlowe,podm_w_rej_reg_na10tys_ludn,udzial_wyrejestr_do_nowych,podm_w_rej_reg_sekt_pryw_HANDL,bud_niemieszk_ogolem,magazyny_silosy_zb
Rok,,,,,,,,,
2005,2245.02,2366.0,122.0,0.0,1234.0,4.3,2224.0,292640.0,115736.0
2006,2362.00,3246.0,179.0,0.0,1271.0,5.0,2431.0,412047.0,93144.0
2007,2596.78,3252.0,197.0,0.0,1304.0,4.5,2685.0,529840.0,250362.0
2008,2901.70,3468.0,231.0,2.0,1345.0,4.5,2925.0,495301.0,296093.0
2009,2909.66,3911.0,227.0,3.0,1374.0,6.5,3187.0,400555.0,208572.0
2010,3161.97,4518.0,322.0,2.0,1429.0,4.4,3527.0,249541.0,70259.0
2011,3313.05,4152.0,335.0,6.0,1435.0,7.5,3832.0,196554.0,56406.0
2012,3405.01,4308.0,377.0,3.0,1474.0,5.2,4229.0,284878.0,130571.0
2013,3513.25,4537.0,565.0,3.0,1503.0,5.8,4781.0,189001.0,55934.0


In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Legend, CustomJS
from bokeh.palettes import Spectral6
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models.widgets import CheckboxGroup, AutocompleteInput
from bokeh.layouts import widgetbox

#output_notebook()


def json_data_one_powiat(powiat):
    
    df = dane_p
    
    #Merge gus dataframes pdf 
    merged = pdf.set_index('TERYT').merge(df.set_index('TERYT'), on='TERYT', how='left') 
    merged = merged[merged.powiat == powiat.lower()]
    print('powiat', powiat)
    print(merged.shape)
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data


def update_data(attr, old, new):
    
    powiat = input_widget.value
    data = wybor_powiatu(powiat)
    
    p_lines.title.text = 'Zmiana poszczególnych wielkości w latach, %s ' %powiat
    
    mypalette=['red', 'blue', 'green', 'black', 'grey', 'purple', 'yellow', 'pink', 'orange']
    ind=0
    legend_it = []
    for col in data.columns:
        source[ind].data = dict(x=data.index, y=data[col])
        ind=ind+1
        
    #Dodanie kolorku na mapie - powiat wybrany w input-okienku do drugiego wykresu
    geosource2 = GeoJSONDataSource(geojson = json_data_one_powiat(powiat))
    p.patches('xs','ys', source = geosource2,line_color = 'firebrick', line_width = 0.7, fill_alpha = 0)       
        
    
        
    

TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
p_lines = figure(title="Zmiana poszczególnych wielkości w latach, powiat Poznań", y_axis_type="linear", plot_height = 800,
           tools = TOOLS, plot_width = 1100)
p_lines.xaxis.axis_label = 'Year'
p_lines.yaxis.axis_label = 'Value'
#p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')



mypalette=['red', 'blue', 'green', 'black', 'grey', 'purple', 'yellow', 'pink', 'orange']
ind=0
legend_it = []
source = {}
c = {}
for col in data.columns:
    source[ind] = ColumnDataSource(data=dict(x=data.index, y=data[col]))
    c[ind] = p_lines.line(x='x', y='y', source=source[ind], line_color=mypalette[ind], line_width = 3, alpha=0.8,
           muted_color=mypalette[ind], muted_alpha=0, name=col)
    legend_it.append((col, [c[ind]]))
    ind=ind+1


legend = Legend(items=legend_it, location=(0, -40))
legend.click_policy="mute"

p_lines.add_layout(legend, 'right')    
    
p_lines.select_one(HoverTool).tooltips = [
    ('Year', '@x'),
    ('Value', '@y'),
    ('Name', '$name')
]

input_widget = AutocompleteInput(completions=list(powiaty_lista), title='Wybierz powiat:')
input_widget.on_change('value', update_data)


code = '''if (cb_data.source.selected.indices.length > 0){
            //lines.visible = true;
            selected_index = cb_data.source.selected.indices[0];
            System.out.print(selected_index)
            //lines.data_source.data['y'] = geosource[selected_index]
            //lines.data_source.change.emit(); 
          }'''


#interakcja z pierwszym rysunkiem
#taptool = p.select(TapTool) #.callback #= print('@powiat') #CustomJS(args = {}, code = code)



#scode = """
#        console.log(cb_obj);
#       console.log(cb_data);
#        """
#taptool.callback = CustomJS(args=dict(source=source),code = scode)

#layout = row(p_lines,widgetbox(input_widget))
#output_file("line_chart.html", title="Line Chart")
#show(layout)
#curdoc().add_root(layout)

In [ ]:
layout = row(
        column(p,widgetbox(slider), 
               widgetbox(select), 
               widgetbox(button)), 
        column(p_lines,
               widgetbox(input_widget))
        )

In [ ]:

curdoc().add_root(layout)

In [ ]:
#Add hover tool
#p.select_one(HoverTool).tooltips = [
#                                    ('powiat',' @powiat'),('wartosc', '@' + col)
#                                ]


a moze po prostu jakos podswietlac ten powiat ktory wybrany na liscie w drugim wykresie?

dobry kod (na samym dole):
    https://stackoverflow.com/questions/48629103/two-interactive-bokeh-plots-select-a-value-in-one-graph-and-change-the-other

### przeczytac!

https://www.programcreek.com/python/example/106841/bokeh.models.TapTool

### CHECK

In [146]:
d = dane_p[['TERYT', 'Nazwa']].groupby('Nazwa').count()

In [147]:
d[d.TERYT>1]

,TERYT
Nazwa,
Powiat bielski,2
Powiat brzeski,2
Powiat grodziski,2
Powiat krośnieński,2
Powiat nowodworski,2
Powiat opolski,2
Powiat ostrowski,2
Powiat tomaszowski,2
Powiat średzki,2


In [148]:
d2 = dane_p[['TERYT', 'Nazwa']].groupby('TERYT').count()

In [150]:
d2 = d2[d2.Nazwa>1]
d2

,Nazwa
TERYT,
0022,2
0200,2
0201,2
0221,2
0261,2


In [160]:
dane_p[dane_p.TERYT == '0261']

,TERYT,Nazwa,wynagrodzenie_srednie_brutto_2008,wynagrodzenie_srednie_brutto_2009,wynagrodzenie_srednie_brutto_2010,wynagrodzenie_srednie_brutto_2011,wynagrodzenie_srednie_brutto_2012,wynagrodzenie_srednie_brutto_2013,wynagrodzenie_srednie_brutto_2014,wynagrodzenie_srednie_brutto_2015,...,gestosc_zaludnienia_na1km2_2009,gestosc_zaludnienia_na1km2_2010,gestosc_zaludnienia_na1km2_2011,gestosc_zaludnienia_na1km2_2012,gestosc_zaludnienia_na1km2_2013,gestosc_zaludnienia_na1km2_2014,gestosc_zaludnienia_na1km2_2015,gestosc_zaludnienia_na1km2_2016,gestosc_zaludnienia_na1km2_2017,gestosc_zaludnienia_na1km2_2018
28,0261,Powiat m.Jelenia Góra,2795.85,2913.03,2969.75,3192.15,3291.09,3367.22,3577.03,3835.12,...,774.0,769.0,764.0,759.0,751.0,745.0,742.0,737.0,733.0,728.0
312,0261,Powiat skarżyski,2821.43,3035.02,2783.02,3001.61,3119.33,3280.83,3395.64,3448.29,...,198.0,201.0,200.0,199.0,197.0,196.0,195.0,193.0,191.0,189.0


In [161]:
dane_p[dane_p.TERYT == '0221']

,TERYT,Nazwa,wynagrodzenie_srednie_brutto_2008,wynagrodzenie_srednie_brutto_2009,wynagrodzenie_srednie_brutto_2010,wynagrodzenie_srednie_brutto_2011,wynagrodzenie_srednie_brutto_2012,wynagrodzenie_srednie_brutto_2013,wynagrodzenie_srednie_brutto_2014,wynagrodzenie_srednie_brutto_2015,...,gestosc_zaludnienia_na1km2_2009,gestosc_zaludnienia_na1km2_2010,gestosc_zaludnienia_na1km2_2011,gestosc_zaludnienia_na1km2_2012,gestosc_zaludnienia_na1km2_2013,gestosc_zaludnienia_na1km2_2014,gestosc_zaludnienia_na1km2_2015,gestosc_zaludnienia_na1km2_2016,gestosc_zaludnienia_na1km2_2017,gestosc_zaludnienia_na1km2_2018
22,0221,Powiat wałbrzyski,2952.00,3144.13,3281.07,3470.48,3564.91,3138.11,3213.78,3293.54,...,349.0,350.0,347.0,345.0,134.0,134.0,133.0,132.0,131.0,130.0
254,0221,Powiat nowodworski,2489.29,2712.39,2751.87,2972.49,3114.10,3209.61,3350.00,3422.12,...,53.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,53.0,53.0


In [162]:
dane_p[dane_p.TERYT == '0201']

,TERYT,Nazwa,wynagrodzenie_srednie_brutto_2008,wynagrodzenie_srednie_brutto_2009,wynagrodzenie_srednie_brutto_2010,wynagrodzenie_srednie_brutto_2011,wynagrodzenie_srednie_brutto_2012,wynagrodzenie_srednie_brutto_2013,wynagrodzenie_srednie_brutto_2014,wynagrodzenie_srednie_brutto_2015,...,gestosc_zaludnienia_na1km2_2009,gestosc_zaludnienia_na1km2_2010,gestosc_zaludnienia_na1km2_2011,gestosc_zaludnienia_na1km2_2012,gestosc_zaludnienia_na1km2_2013,gestosc_zaludnienia_na1km2_2014,gestosc_zaludnienia_na1km2_2015,gestosc_zaludnienia_na1km2_2016,gestosc_zaludnienia_na1km2_2017,gestosc_zaludnienia_na1km2_2018
2,0201,Powiat bolesławiecki,2476.41,2629.01,2789.80,2936.92,3057.28,3231.49,3422.90,3548.69,...,68.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0,69.0
236,0201,Powiat siemiatycki,2402.98,2536.47,2670.09,2765.25,2948.59,3016.98,3157.37,3220.48,...,32.0,33.0,33.0,32.0,32.0,32.0,32.0,31.0,31.0,31.0


In [194]:
df = dane_p
    
#Merge gus dataframes pdf 
merged = pdf.set_index('TERYT').merge(df.set_index('TERYT'), on='TERYT', how='left') 
#merged = merged[merged.powiat == powiat]
print(merged.shape)
    

(387, 133)


In [200]:
sum(merged.powiat == 'Powiat wołomiński'.lower())

1